# Introdução ao Deep Learning

In [1]:
!pip install tensorflow

In [2]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/Universidade de dados/Semana 17: Introdução ao Deep Learning/cereal.csv')
df.head()

,name,mfr,type,calories,protein,fat,sodium,fiber,carbo,sugars,potass,vitamins,shelf,weight,cups,rating
0,100% Bran,N,C,70,4,1,130,10.0,5.0,6,280,25,3,1.0,0.33,68.402973
1,100% Natural Bran,Q,C,120,3,5,15,2.0,8.0,8,135,0,3,1.0,1.00,33.983679
2,All-Bran,K,C,70,4,1,260,9.0,7.0,5,320,25,3,1.0,0.33,59.425505
3,All-Bran with Extra Fiber,K,C,50,4,0,140,14.0,8.0,0,330,25,3,1.0,0.50,93.704912
4,Almond Delight,R,C,110,2,2,200,1.0,14.0,8,-1,25,3,1.0,0.75,34.384843


In [3]:
df.shape

(77, 16)

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77 entries, 0 to 76
Data columns (total 16 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   name      77 non-null     object 
 1   mfr       77 non-null     object 
 2   type      77 non-null     object 
 3   calories  77 non-null     int64  
 4   protein   77 non-null     int64  
 5   fat       77 non-null     int64  
 6   sodium    77 non-null     int64  
 7   fiber     77 non-null     float64
 8   carbo     77 non-null     float64
 9   sugars    77 non-null     int64  
 10  potass    77 non-null     int64  
 11  vitamins  77 non-null     int64  
 12  shelf     77 non-null     int64  
 13  weight    77 non-null     float64
 14  cups      77 non-null     float64
 15  rating    77 non-null     float64
dtypes: float64(5), int64(8), object(3)
memory usage: 9.8+ KB


In [4]:
df.nunique().sort_values(ascending=False)

name        77
rating      77
potass      36
sodium      27
carbo       22
sugars      17
fiber       13
cups        12
calories    11
mfr          7
weight       7
protein      6
fat          5
vitamins     3
shelf        3
type         2
dtype: int64

Vamos tentar prever a quantidade de calorias com base em algumas features especificas desse df

In [5]:
features = [
    'protein',
    'fat',
    'carbo',
    'weight',
    'cups'
]

X = df[features]
y = df['calories'] # Target

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

# Padronização depois do split pra nao ter vazamento dos dados teste
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train) # Treina o scaler com o treino
X_test = scaler.transform(X_test) # Aplica-se o que foi treinado no X_Train

Nos vamos criar nossa rede neural com keras.sequential e dentro dele vamos criar algunas camadas densas com layers.Dense, e passamos como parametro quantos neuronios estarao la e quantas features terá de entrada

In [7]:
from tensorflow import keras
from tensorflow.keras import layers

# Rede neural com 1 neuronio
modelo = keras.Sequential([
    layers.Dense(units=1, input_shape=[len(features)])
])


In [8]:
pesos, vies = modelo.weights
print(f'Pesos: {pesos}, Vies: {vies}')
# Como ainda nao treinamos o modelo, ele vai retornar os pesos que foram inicalizados aleatoriamente

Pesos: <tf.Variable 'dense/kernel:0' shape=(5, 1) dtype=float32, numpy=
array([[ 0.9121702 ],
       [ 0.15716076],
       [ 0.68350434],
       [-0.8420558 ],
       [ 0.25575972]], dtype=float32)>, Vies: <tf.Variable 'dense/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>


Para treinar o nosso modelo precisamos de duas coisas:
- Loss function: Para quantificar o quao bom nosso modelo está indo
- Otimizador: Diz a rede neural como atualizar os pesos

In [9]:
# Adicionando um otimizador e uma função perda ao nosso modelo
modelo.compile(
    optimizer='adam', #  algoritmo do gradiente descendente estocástico
    loss='mae' # iremos enxergar este problema sob a ótica de um problema de regressão
)

Treinando -- finalmente -- o modelo

In [10]:
historico = modelo.fit(
    x=X_train,
    y=y_train,
    validation_data=(X_test, y_test),
    batch_size = X_train.shape[0], # Numero de linhas do teste
    epochs = 10 # Nummero de iretações
)

Epoch 1/10
1/1 [==============================] - 1s 797ms/step - loss: 108.6792 - val_loss: 102.4362
Epoch 2/10
1/1 [==============================] - 0s 40ms/step - loss: 108.6783 - val_loss: 102.4353
Epoch 3/10
1/1 [==============================] - 0s 40ms/step - loss: 108.6772 - val_loss: 102.4343
Epoch 4/10
1/1 [==============================] - 0s 31ms/step - loss: 108.6763 - val_loss: 102.4333
Epoch 5/10
1/1 [==============================] - 0s 33ms/step - loss: 108.6752 - val_loss: 102.4323
Epoch 6/10
1/1 [==============================] - 0s 34ms/step - loss: 108.6742 - val_loss: 102.4313
Epoch 7/10
1/1 [==============================] - 0s 35ms/step - loss: 108.6732 - val_loss: 102.4304
Epoch 8/10
1/1 [==============================] - 0s 31ms/step - loss: 108.6722 - val_loss: 102.4294
Epoch 9/10
1/1 [==============================] - 0s 28ms/step - loss: 108.6712 - val_loss: 102.4284
Epoch 10/10
1/1 [==============================] - 0s 32ms/step - loss: 108.6702 - val_los

Em cada iteração, o algoritmo a partir do gradiente descendente realiza uma otimização e mostra os resultados nos dados de treino e teste na loss function que ecolhemos

# Deep Learning

Vamos aplicar nosso novo modelo de deep learning de regressão  e usando alguns novos conhecimentos

In [23]:
features = [
    'protein',
    'fat',
    'carbo',
    'weight',
    'cups'
]

X = df[features]
y = df['calories'] # Target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

- Relu como activation Funcion evita que o resultado seja uma combinação linear dos pesos e dos inputs substituindo valores neativos por 0, pra fazer a rede neural pegar padrões nao lineares
- softmax  converte o output final em uma probabilidade entre 0 e 1, que representa a probabilidade do resultado ser de uma classe
- Early Stopping faz com que o modelo pare de aprender quando começa a overfittar
- DropOut faz com que alguns neuronios nao participem de algumas iterações do treinamento para evitar decorar os ruidos
- Batch normalization normaliza os dados durante o reinamento

In [24]:
modelo = keras.Sequential(layers=[
    layers.BatchNormalization(), # Normalização durante o reinamento
    layers.Dropout(rate=0.2), # 20% dos neuronios nao vai participar

    layers.Dense(units=64, activation='relu', input_shape=[len(features)]), # 1 Hidden

    layers.BatchNormalization(), # Normalização durante o reinamento
    layers.Dropout(rate=0.2), # 20% dos neuronios nao vai participar

    layers.Dense(units=16, activation='relu'), # 2 Hidden

    layers.Dense(units=1)  # Output
])

In [25]:
# Compilando o modelo para classificação
modelo.compile(
    optimizer = 'adam', # Uma forma de gradiente descendente
    loss = 'mean_squared_error', # Loss fucntion para classificação com base na probabilidade das classes
    metrics = ['mae'] # Baseado na acuracia
)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(
    min_delta = 0.001, # Variação minima que é considerada melhoria
    patience = 20, # Epocas para esperar ate parar o treinamento
    restore_best_weights = True # QUando perceber overfit voltar para os menores parametros
)

# Treinando o novo modelo com o early stopping erly
history = modelo.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    batch_size = 256, epochs = 100,
    callbacks = [early_stopping] # Parar quando começar a overfittar
)